In [ ]:
# Colab environment setup
import numpy as np
# Install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conf

In [ ]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

In [ ]:
import esm

In [ ]:
# !pip install h5py
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=27ab151071c249d71f50ae55f328fdc2ced7eab9905d515793bc1f5af2bce461
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import h5py

In [ ]:
# Let's start with google disk folder with preprocessed files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATADIR = Path("/content/gdrive/MyDrive/MISATO-experiments")


In [ ]:
WORKDIR = Path("/content/workdir")
WORKDIR.mkdir(exist_ok=True)
INPUTDIR = DATADIR / "preprocessed"
ESMIFDIR = DATADIR / "esm-if-embeddings"
ESMIFDIR.mkdir(exist_ok=True)
ESM2DIR = DATADIR / "esm2-embeddings"
ESM2DIR.mkdir(exist_ok=True)

In [ ]:
df = pd.read_csv(INPUTDIR/"misato_sequences_info.csv")
sequences = df.sequence.unique()  #.shape, df.shape

In [ ]:
from tqdm.auto import tqdm

/content


In [ ]:

from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('facebook/esm2_t33_650M_UR50D')
model = SentenceTransformer('facebook/esm2_t33_650M_UR50D')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/facebook_esm2_t33_650M_UR50D were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/facebook_esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN t

In [ ]:
from tqdm.auto import tqdm


# esm2_t6_8M_UR50D

batch_size = 16*8*2*2
embeddings = []
for i in tqdm(range(0, len(sequences), batch_size), total=len(sequences)//batch_size+1):
  seq_subset = sequences[i:i+batch_size]
  embeddings.append(model.encode(seq_subset))
  # if i >2:
  #   break

embeddings = np.concatenate(embeddings)

np.savez_compressed(ESM2DIR / "embeddings_esm2_650M.npz", embeddings)
with open("emb_sequences.txt", 'w') as f:
  for sequence in sequences:
    f.write(sequence +"\n")


  0%|          | 0/59 [00:00<?, ?it/s]

In [ ]:
del model

In [ ]:
model = SentenceTransformer('facebook/esm2_t6_8M_UR50D')

# esm2_t6_8M_UR50D

batch_size = 16*8*4
embeddings = []
for i in tqdm(range(0, len(sequences), batch_size), total=len(sequences)//batch_size+1):
  seq_subset = sequences[i:i+batch_size]
  embeddings.append(model.encode(seq_subset))
  # if i >2:
  #   break

embeddings = np.concatenate(embeddings)

np.savez_compressed(ESM2DIR / "embeddings_esm2_8M.npz", embeddings)
with open("emb_sequences2.txt", 'w') as f:
  for sequence in sequences:
    f.write(sequence +"\n")

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/facebook_esm2_t6_8M_UR50D were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/facebook_esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this mo

  0%|          | 0/59 [00:00<?, ?it/s]